In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

from sklearn.metrics import confusion_matrix

In [3]:
test_df = pd.read_csv('test.csv')

In [5]:
train_df = pd.read_csv('train.csv')

In [7]:
train_df.shape

(42000, 785)

In [11]:
test_df.shape

(28000, 784)